In [2]:
import pandas as pd

In [6]:
pd.read_csv('./sql_interview_questions.csv', sep=';')

,id,question,answer,category,difficulty_level,tags,example_query,explanation,common_mistakes,related_questions
0,499,"""You are given two tables: 'sales' and 'produc...","""SELECT p.product_name, COALESCE(SUM(s.quantit...",""".""Data Analyst""","""Intermediate""","""Data Analyst, Join, Aggregation, SQL Functions""","""SELECT p.product_name, COALESCE(SUM(s.quantit...","""","""This query uses a LEFT JOIN to ensure all pro...","""Candidates might forget to use COALESCE, resu..."
1,500,"""You are given two tables: 'Sales' and 'Produc...","""WITH ranked_sales AS ( SELECT p.category, p.p...","""","""Data Analysis""","""Advanced""","""Data Analyst, Join, Aggregate function, Windo...","""WITH ranked_sales AS ( SELECT p.category, p.p...","""","""The query first creates a Common Table Expres..."
2,503,"""Given two tables, 'employees' (emp_id, emp_na...","""SELECT e.emp_name, d.dept_name FROM employee...","""","""Data Analyst""","""Intermediate""","""Data Analyst, Join, Aggregate function""","""N/A""","""This query uses a LEFT JOIN to find employees...","""Candidates might overlook the need for filte..."
3,504,"""Given a sales database with two tables: 'Sale...","""SELECT p.ProductID, p.ProductName, SUM(s.Quan...","""","""Sales Analysis""","""Intermediate""","""Data Analyst, Aggregate function, Join, Group...","""SELECT p.ProductID, p.ProductName, SUM(s.Quan...","""","""This query joins the Sales and Products table..."
4,1,"""You are tasked with analyzing sales data from...","""SELECT p.category, SUM(s.amount) AS total_sal...","""","""SQL""","""Intermediate""","[""Data Analyst"", ""Business Intelligence"", ""Joi...","""""","""This query joins the sales and products tabl...","""Candidates might mistakenly forget to filter..."
...,...,...,...,...,...,...,...,...,...,...
597,606,"In a 'subscriptions' table with 'user_id', 'st...",SELECT * FROM subscriptions WHERE end_date IS ...,SQL,Easy,"Data Analyst, Subscription Manager",SELECT * FROM subscriptions WHERE end_date IS ...,This retrieves current subscriptions for users.,Confusing active with expired subscriptions.,What are retention metrics?; How to evaluate c...
598,607,Write a SQL query to find all customers who pu...,"SELECT customer_id, product_id, COUNT(*) AS pu...",SQL,Intermediate,"Data Analyst, Marketing Analyst","SELECT customer_id, product_id, COUNT(*) AS pu...",This highlights repeat customers for specific ...,Not accounting for the grouping correctly.,What are customer loyalty metrics?; How to eva...
599,608,Assuming you have a 'visits' table with 'user_...,SELECT SUM(duration) AS total_time FROM visits...,SQL,Intermediate,"Data Analyst, Marketing Analyst",SELECT SUM(duration) AS total_time FROM visits...,This sums total visit time for the current month.,Neglecting to group or filter for the right date.,What are engagement metrics?; How to analyze u...
600,609,Write a query to find product categories with ...,SELECT DISTINCT category FROM products WHERE q...,SQL,Intermediate,"Inventory Manager, Data Analyst",SELECT DISTINCT category FROM products WHERE q...,This identifies categories lacking stock.,Misunderstanding groupings for empty categories.,What are stock management strategies?; How to ...
